In [ ]:
from dotenv import dotenv_values
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain_openai import ChatOpenAI

config = dotenv_values(".env")

chat = ChatOpenAI(
    openai_api_key=config["OPENAI_API_KEY"],
    model="gpt-4o-mini",
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=200,
    return_messages=True,
    memory_key="history",
)

examples = [
    {"movie": "탑건 (Top Gun)", "answer": "🛩️👨‍✈️🔥"},
    {"movie": "대부 (God Father)", "answer": "👨‍👨‍👦🔫🍝"},
    {"movie": "오펜하이머 (Oppenheimer)", "answer": "🔱🎆🛫"},
    {"movie": "아바타 (Avatar)", "answer": "💦🪂🏝"},
]
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Express {movie} as three emojis."),
        ("ai", "{answer}"),
    ]
)
prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are movie lover that can express it as three emojis."),
        MessagesPlaceholder(variable_name="history"),
        prompt,
        ("human", "{movie}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | chat


def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    # print(result.content)

In [38]:
invoke_chain("매트릭스 (The Matrix)")

💊🕶️💻

In [39]:
invoke_chain("타이타닉 (Titanic)")

🚢💔🌊

In [40]:
invoke_chain("인셉션 (Inception)")

💤🌌🌀

In [41]:
invoke_chain("반지의제왕 (The Lord of the Ring)")

💍🧙‍♂️🗡️

In [42]:
invoke_chain("쥬라기공원 (Jurassic Park)")

🦖🌴🚪

In [43]:
load_memory(_)

[HumanMessage(content='매트릭스 (The Matrix)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='💊🕶️💻', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='타이타닉 (Titanic)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='🚢💔🌊', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='인셉션 (Inception)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='💤🌌🌀', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반지의제왕 (The Lord of the Ring)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='💍🧙\u200d♂️🗡️', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='쥬라기공원 (Jurassic Park)', additional_kwargs={}, response_metadata={}),
 AIMessage(content='🦖🌴🚪', additional_kwargs={}, response_metadata={})]